In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.11.1',
 'qiskit-aer': '0.3.4',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.4.5',
 'qiskit-aqua': '0.6.2',
 'qiskit': '0.14.1'}

## Molecule set-up

In [2]:
import numpy as np
import pylab
from pyscf import gto, scf, dft
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import ExactEigensolver, VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.core import Hamiltonian, QubitMappingType, TransformationType
from qiskit.aqua.input import EnergyInput

## initialize some params for the outer loop - overall analysis
#energies = np.empty([len(algorithms), steps+1])
#hf_energies = np.empty(steps+1)
#distances = np.empty(steps+1)
#aqua_globals.random_seed = 750
#steps = 0

algorithms = [{'name': 'VQE'},
              {'name': 'ExactEigensolver'}]
molecule = 'H .0 .0 -{0}; H .0 .0 {0}'
driver = PySCFDriver(molecule.format(0.25), basis='sto3g')

###
### build molecule object in Qiskit
###
qmolecule = driver.run()

In [4]:
#len(qmolecule.mo_coeff)

qmolecule.mo_coeff
#qmolecule.one_body_integrals
#qmolecule.two_body_integrals
#qmolecule.hf_energy
#qmolecule.nuclear_repulsion_energy
#qmolecule.nuclear_dipole_moment
#qmolecule.reverse_dipole_sign
#qmolecule.core_orbitals
#qmolecule.has_dipole_integrals
qmolecule.num_orbitals
#dir(qmolecule)

array([[ 9.47964642e-01, -3.84542230e-01, -5.20455239e-02,
         1.35012604e-17, -7.80874785e-17, -7.32739848e-01],
       [-3.53908877e-02,  5.80910786e-01,  7.97379180e-01,
        -1.03802989e-16,  2.17724005e-16, -9.44433506e-01],
       [ 1.83389776e-19,  2.61628519e-18, -4.15373424e-16,
         1.61935811e-01,  9.86801294e-01,  1.56558967e-18],
       [-0.00000000e+00, -0.00000000e+00,  1.11022302e-16,
         9.86801294e-01, -1.61935811e-01, -0.00000000e+00],
       [-1.46091296e-02,  4.84887781e-01, -6.52892392e-01,
         1.34394319e-17, -5.35797608e-16, -8.81410256e-01],
       [ 1.08475456e-01,  3.73570666e-01, -2.33787311e-01,
         2.15181517e-17,  5.58595507e-17,  1.69556060e+00]])

6

### No Orbital Reduction, Depth = 3, Full Entanglement

In [7]:
operator =  Hamiltonian(transformation=TransformationType.FULL, # full or particle_hole
                        qubit_mapping=QubitMappingType.PARITY,  # jordan_wigner, parity or bravyi_kitaev
                        two_qubit_reduction=True)#,
qubit_op, aux_ops = operator.run(qmolecule)

#var_form = RYRZ(qubit_op.num_qubits, depth=3, entanglement='full')
#var_circ = var_form.construct_circuit(np.zeros(var_form.num_parameters))

#print("Num qubits = ", qubit_op.num_qubits)
#print("Num gates = ", len(var_circ))
#var_circ.draw(output='mpl')


full
parity
True


False

In [10]:
operator._freeze_core
qmolecule.core_orbitals
operator._orbital_reduction

False

[0]

[]

#### Inspecting 'run' function of class Hamiltonian

In [61]:
self_hf_energy = qmolecule.hf_energy
self_nuclear_repulsion_energy = qmolecule.nuclear_repulsion_energy
self_nuclear_dipole_moment = qmolecule.nuclear_dipole_moment
self_reverse_dipole_sign = qmolecule.reverse_dipole_sign
self_freeze_core = operator._freeze_core
self_orbital_reduction = operator._orbital_reduction
core_list = qmolecule.core_orbitals if self_freeze_core else []
reduce_list = self_orbital_reduction
transformation = 'full'
qubit_mapping = 'parity'
two_qubit_reduction = True

if self_freeze_core:
    logger.info("Freeze_core specified. Core orbitals to be frozen: %s", core_list)
if reduce_list:
    logger.info("Configured orbital reduction list: %s", reduce_list)
    reduce_list = [x + qmolecule.num_orbitals if x < 0 else x for x in reduce_list]

freeze_list = []
remove_list = []

orbitals_list = list(set(core_list + reduce_list))
num_alpha = qmolecule.num_alpha
num_beta = qmolecule.num_beta
new_num_alpha = num_alpha
new_num_beta = num_beta

In [15]:
if orbitals_list:
    orbitals_list = np.array(orbitals_list)
    orbitals_list = orbitals_list[(orbitals_list >= 0) &
                                          (orbitals_list < qmolecule.num_orbitals)]

    freeze_list_alpha = [i for i in orbitals_list if i < num_alpha]
    freeze_list_beta = [i for i in orbitals_list if i < num_beta]
    freeze_list = np.append(freeze_list_alpha,
                                    [i + qmolecule.num_orbitals for i in freeze_list_beta])

    remove_list_alpha = [i for i in orbitals_list if i >= num_alpha]
    remove_list_beta = [i for i in orbitals_list if i >= num_beta]
    rla_adjust = -len(freeze_list_alpha)
    rlb_adjust = -len(freeze_list_alpha) - len(freeze_list_beta) + qmolecule.num_orbitals
    remove_list = np.append([i + rla_adjust for i in remove_list_alpha],
                                    [i + rlb_adjust for i in remove_list_beta])

    new_num_alpha -= len(freeze_list_alpha)
    new_num_beta -= len(freeze_list_beta)

new_nel = [new_num_alpha, new_num_beta]

In [51]:
new_nel
freeze_list
remove_list

[2, 2]

[]

[]

In [18]:
from qiskit.chemistry.fermionic_operator import FermionicOperator
fer_op = FermionicOperator(h1=qmolecule.one_body_integrals, h2=qmolecule.two_body_integrals)
# Args for FermionicOperator:
#           h1 (numpy.ndarray): second-quantized fermionic one-body operator, a 2-D (NxN) tensor
#           h2 (numpy.ndarray): second-quantized fermionic two-body operator,
#                               a 4-D (NxNxNxN) tensor
#           ph_trans_shift (float): energy shift caused by particle hole transformation

In [20]:
 #'_bravyi_kitaev_mode',  [function that needs to be invoked with input]
 #'_convert_to_block_spins',
 #'_convert_to_interleaved_spins',
 #'_h1_transform',   [function that needs to be invoked with input]
 #'_h2_transform',   [function that needs to be invoked with input]
 #'_jordan_wigner_mode', [function that needs to be invoked with input]
 #'_modes', [set in __init__ from inputs]
 #'_one_body_mapping',  [function that needs to be invoked with input]
 #'_parity_mode', [function that needs to be invoked with input]
 #'_two_body_mapping',   [invoked in 'mapping' function]
 #'fermion_mode_elimination', [function that needs to be invoked with input]
 #'fermion_mode_freezing',    [function that needs to be invoked with input]
 #'mapping', [function that needs to be invoked with input]
 #'particle_hole_transformation',   [function that needs to be invoked with input]
 #'total_angular_momentum',
 #'total_magnetization',
 #'total_particle_number',
 #'transform'

 #'_s_x_squared',
 #'_s_y_squared',
 #'_s_z_squared',
 #'h1',
 #'h2',

In [53]:
fer_op, _energy_shift, did_shift = Hamiltonian._try_reduce_fermionic_operator(fer_op, freeze_list, remove_list)

fer_op
_energy_shift
did_shift

0.0

False

In [66]:
if did_shift:
    logger.info("Frozen orbital energy shift: %s", self._energy_shift)
    
if transformation == 'particle_hole':
    fer_op, ph_shift = fer_op.particle_hole_transformation(new_nel)
    self._ph_energy_shift = -ph_shift
    logger.info("Particle hole energy shift: %s", self._ph_energy_shift)

print('Converting to qubit using %s mapping', qubit_mapping)
qubit_op = Hamiltonian._map_fermionic_operator_to_qubit(fer_op, qubit_mapping, new_nel, two_qubit_reduction)

Converting to qubit using %s mapping parity


In [68]:
dir(qubit_op)

['__abstractmethods__',
 '__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_or_sub',
 '_aer_paulis',
 '_atol',
 '_basis',
 '_eval_directly',
 '_name',
 '_paulis',
 '_paulis_table',
 '_routine_compute_mean_and_var',
 '_scaling_weight',
 '_simplify_paulis',
 '_to_dia_matrix',
 '_z2_symmetries',
 'add',
 'aer_paulis',
 'anticommute_with',
 'atol',
 'basis',
 'chop',
 'coloring',
 'commute_with',
 'construct_evaluation_circuit',
 'construct_evolution_circuit',
 'convert',
 'copy',
 'disable_summarize_circuits',
 'enable_summarize_circuits',
 'eval',
 'evalua

In [ ]:
print('  num paulis: %s, num qubits: %s', len(qubit_op.paulis), qubit_op.num_qubits)
algo_input = EnergyInput(qubit_op)

def _add_aux_op(aux_op):
    algo_input.add_aux_op(Hamiltonian._map_fermionic_operator_to_qubit(aux_op, qubit_mapping, new_nel, two_qubit_reduction))
    print('  num paulis: %s', len(algo_input.aux_ops[-1].paulis))

print('Creating aux op for Number of Particles')
_add_aux_op(fer_op.total_particle_number())

print('Creating aux op for S^2')
_add_aux_op(fer_op.total_angular_momentum())

print('Creating aux op for Magnetization')
_add_aux_op(fer_op.total_magnetization())

In [ ]:
if qmolecule.has_dipole_integrals():
    def _dipole_op(dipole_integrals, axis):
        logger.debug('Creating aux op for dipole %s', axis)
        fer_op_ = FermionicOperator(h1=dipole_integrals)
        fer_op_, shift, did_shift_ = self._try_reduce_fermionic_operator(fer_op_,
                                                                                 freeze_list,
                                                                                 remove_list)
        if did_shift_: 
            logger.info("Frozen orbital %s dipole shift: %s", axis, shift)
            ph_shift_ = 0.0
        if self._transformation == TransformationType.PARTICLE_HOLE.value:
            fer_op_, ph_shift_ = fer_op_.particle_hole_transformation(new_nel)
            ph_shift_ = -ph_shift_
            logger.info("Particle hole %s dipole shift: %s", axis, ph_shift_)
        qubit_op_ = self._map_fermionic_operator_to_qubit(fer_op_,
                                                                  self._qubit_mapping,
                                                                  new_nel,
                                                                  self._two_qubit_reduction)
        logger.debug('  num paulis: %s', len(qubit_op_.paulis))
        return qubit_op_, shift, ph_shift_

    op_dipole_x, self._x_dipole_shift, self._ph_x_dipole_shift = \
                _dipole_op(qmolecule.x_dipole_integrals, 'x')
    op_dipole_y, self._y_dipole_shift, self._ph_y_dipole_shift = \
                _dipole_op(qmolecule.y_dipole_integrals, 'y')
    op_dipole_z, self._z_dipole_shift, self._ph_z_dipole_shift = \
                _dipole_op(qmolecule.z_dipole_integrals, 'z')

    algo_input.add_aux_op(op_dipole_x)
    algo_input.add_aux_op(op_dipole_y)
    algo_input.add_aux_op(op_dipole_z)

logger.info('Molecule num electrons: %s, remaining for processing: %s',
                    [num_alpha, num_beta], new_nel)
nspinorbs = qmolecule.num_orbitals * 2
new_nspinorbs = nspinorbs - len(freeze_list) - len(remove_list)
logger.info('Molecule num spin orbitals: %s, remaining for processing: %s',
                    nspinorbs, new_nspinorbs)

self._add_molecule_info(self.INFO_NUM_PARTICLES, [new_num_alpha, new_num_beta])
self._add_molecule_info(self.INFO_NUM_ORBITALS, new_nspinorbs)
self._add_molecule_info(self.INFO_TWO_QUBIT_REDUCTION,
                                self._two_qubit_reduction
                                if self._qubit_mapping == 'parity' else False)

logger.debug('Processing complete ready to run algorithm')
return algo_input.qubit_op, algo_input.aux_ops

### No Orbital Reduction, Depth = 5, Full Entanglement

In [26]:
var_form = RYRZ(qubit_op.num_qubits, depth=5, entanglement='full')
var_circ = var_form.construct_circuit(np.zeros(var_form.num_parameters))
print("Num qubits = ", qubit_op.num_qubits)
print("Num gates = ", len(var_circ))
#var_circ.draw(output='mpl')

Num qubits =  10
Num gates =  806


### No Orbital Reduction, Depth = 3, Linear Entanglement

In [27]:
var_form = RYRZ(qubit_op.num_qubits, depth=3, entanglement='linear')
var_circ = var_form.construct_circuit(np.zeros(var_form.num_parameters))
print("Num qubits = ", qubit_op.num_qubits)
print("Num gates = ", len(var_circ))
#var_circ.draw(output='mpl')

Num qubits =  10
Num gates =  168


### No Orbital Reduction, Depth = 5, Linear Entanglement

In [28]:
var_form = RYRZ(qubit_op.num_qubits, depth=5, entanglement='linear')
var_circ = var_form.construct_circuit(np.zeros(var_form.num_parameters))
print("Num qubits = ", qubit_op.num_qubits)
print("Num gates = ", len(var_circ))
#var_circ.draw(output='mpl')

Num qubits =  10
Num gates =  266


### Orbital Reduction, Depth = 3, Full Entanglement

In [5]:
#operator =  Hamiltonian(transformation=TransformationType.FULL, # full or particle_hole
#                        qubit_mapping=QubitMappingType.PARITY,  # jordan_wigner, parity or bravyi_kitaev
#                        two_qubit_reduction=True),
#                        orbital_reduction=[-3, -2])
    
operator =  Hamiltonian(transformation=TransformationType.FULL,
                        qubit_mapping=QubitMappingType.PARITY,
                        two_qubit_reduction=True,
                        freeze_core=True,
                        orbital_reduction=[-3, -2])
qubit_op, aux_ops = operator.run(qmolecule)
var_form = RYRZ(qubit_op.num_qubits, depth=3, entanglement='full')
var_circ = var_form.construct_circuit(np.zeros(var_form.num_parameters))
optimizer = SPSA(max_trials=200)
algo = VQE(qubit_op, var_form, optimizer)
result = algo.run(QuantumInstance(BasicAer.get_backend('qasm_simulator'), seed_simulator=aqua_globals.random_seed,
                                  seed_transpiler=aqua_globals.random_seed, shots=1))

            
lines, result = operator.process_algorithm_result(result)
print("Num qubits = ", qubit_op.num_qubits)
print("Num gates = ", len(var_circ))
#var_circ.draw(output='mpl')

Num qubits =  4
Num gates =  93


In [15]:

qubit_op.num_qubits
#dir(qubit_op)
#dir(var_circ)
#var_circ.swap
#var_circ.__dict__
len(var_circ)

-7.0284100945419326

3.1750632655200004

array([ 0.        ,  0.        , -0.94486306])

True

[0]

10

492

## VQE part

In [ ]:
#optimizer = SPSA(max_trials=200)
#algo = VQE(qubit_op, var_form, optimizer)
#result = algo.run(QuantumInstance(BasicAer.get_backend('qasm_simulator'), seed_simulator=aqua_globals.random_seed,
#                                  seed_transpiler=aqua_globals.random_seed, shots=1))
     #   else:
     #        result = ExactEigensolver(qubit_op).run()
            
#lines, result = operator.process_algorithm_result(result)
    #energies[j][i] = result['energy']
    #hf_energies[i] = result['hf_energy']
    #distances[i] = d
#print(' --- complete')

#print('Distances: ', distances)
#print('Energies:', energies)
#print('Hartree-Fock energies:', hf_energies)

In [13]:
#pylab.plot(distances, hf_energies, label='Hartree-Fock')
#for j in range(len(algorithms)):
#    pylab.plot(distances, energies[j], label=algorithms[j])
#pylab.xlabel('Interatomic distance (Angstrom)')
#pylab.ylabel('Energy (Hartree)')
#pylab.title('H2 Ground State Energy')
#pylab.legend(loc='upper right');

In [12]:
#pylab.plot(distances, np.subtract(hf_energies, energies[1]), label='Hartree-Fock')
#pylab.plot(distances, np.subtract(energies[0], energies[1]), label=algorithms[0])
#pylab.xlabel('Interatomic distance (Angstrom)')
#pylab.ylabel('Energy (Hartree)')
#pylab.title('Energy difference from ExactEigensolver')
#pylab.legend(loc='upper left');